In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# PDF QA System - 開發測試筆記本\n",
    "\n",
    "這個筆記本用於測試和開發 PDF 問答系統的各個組件。"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 導入必要的模組\n",
    "import sys\n",
    "import os\n",
    "sys.path.append('..')\n",
    "\n",
    "from src.config import Config\n",
    "from src.file_handler import FileHandler\n",
    "from src.ocr_reader import OCRReader\n",
    "from src.summarizer import Summarizer\n",
    "from src.vector_store import VectorStore\n",
    "from src.qa_service import QAService"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. 測試 OCR 功能"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 測試 OCR Reader\n",
    "ocr_reader = OCRReader()\n",
    "\n",
    "# 假設你有一個測試 PDF 檔案\n",
    "pdf_path = '../data/pdfs/test.pdf'  # 請替換為實際檔案路徑\n",
    "\n",
    "if os.path.exists(pdf_path):\n",
    "    text = ocr_reader.extract_text_from_pdf(pdf_path)\n",
    "    print(f\"提取的文字長度: {len(text)}\")\n",
    "    print(f\"前 500 個字符: {text[:500]}...\")\nelse:\n    print(\"測試檔案不存在，請上傳一個 PDF 檔案到 data/pdfs/ 目錄\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. 測試摘要功能"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 測試 Summarizer\n",
    "summarizer = Summarizer()\n",
    "\n",
    "# 使用示例文本\n",
    "sample_text = \"\"\"\n",
    "人工智慧（AI）是電腦科學的一個分支，致力於創建能夠執行通常需要人類智慧的任務的系統。\n",
    "這些任務包括學習、推理、問題解決、感知和語言理解。AI 的發展歷史可以追溯到 20 世紀 50 年代，\n",
    "當時研究人員開始探索機器是否能夠模擬人類思維。近年來，由於深度學習和大數據的發展，\n",
    "AI 技術取得了顯著進展。現在 AI 被廣泛應用於各個領域，包括醫療、金融、交通和娛樂等。\n",
    "\"\"\"\n",
    "\n",
    "summary_path, summary = summarizer.create_summary(sample_text, \"test_document.pdf\")\n",
    "print(f\"摘要: {summary}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. 測試向量存儲功能"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 測試 Vector Store\n",
    "vector_store = VectorStore()\n",
    "\n",
    "# 添加測試文檔\n",
    "test_text = \"\"\"\n",
    "機器學習是人工智慧的一個子領域，它使電腦能夠在沒有明確編程的情況下學習。\n",
    "監督學習是機器學習的一種方法，使用標記的訓練數據來學習從輸入到輸出的映射。\n",
    "無監督學習則是從未標記的數據中發現隱藏的模式或結構。\n",
    "深度學習是機器學習的一個分支，使用多層神經網絡來模擬人腦的工作方式。\n",
    "\"\"\"\n",
    "\n",
    "# 添加文檔到向量存儲\n",
    "success = vector_store.add_document(test_text, \"machine_learning_intro.pdf\")\n",
    "print(f\"文檔添加成功: {success}\")\n",
    "\n",
    "# 搜索測試\n",
    "results = vector_store.search(\"什麼是深度學習？\", top_k=3)\n",
    "print(f\"搜索結果數量: {len(results)}\")\n",
    "for i, result in enumerate(results):\n",
    "    print(f\"結果 {i+1}: {result['content'][:100]}...\")\n",
    "    print(f\"距離: {result['distance']}\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. 測試問答服務"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 測試 QA Service\n",
    "qa_service = QAService()\n",
    "\n",
    "# 測試問題\n",
    "questions = [\n",
    "    \"什麼是機器學習？\",\n",
    "    \"監督學習和無監督學習有什麼區別？\",\n",
    "    \"深度學習如何工作？\"\n",
    "]\n",
    "\n",
    "for question in questions:\n",
    "    print(f\"\\n問題: {question}\")\n",
    "    result = qa_service.answer_question(question)\n",
    "    print(f\"回答: {result['answer']}\")\n",
    "    print(f\"信心度: {result['confidence']}\")\n",
    "    print(f\"來源: {[s['filename'] for s in result['sources']]}\")\n",
    "    print(\"-\" * 50)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. 系統整合測試"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# 完整流程測試\n",
    "def test_full_pipeline(pdf_path):\n",
    "    \"\"\"\n",
    "    測試完整的 PDF 處理和問答流程\n",
    "    \"\"\"\n",
    "    if not os.path.exists(pdf_path):\n",
    "        print(f\"檔案不存在: {pdf_path}\")\n",
    "        return\n",
    "    \n",
    "    filename = os.path.basename(pdf_path)\n",
    "    print(f\"開始處理: {filename}\")\n",
    "    \n",
    "    # 1. OCR 處理\n",
    "    print(\"1. 進行 OCR 處理...\")\n",
    "    ocr_reader = OCRReader()\n",
    "    ocr_path, text = ocr_reader.process_pdf(pdf_path, filename)\n",
    "    \n",
    "    if not text:\n",
    "        print(\"OCR 處理失敗\")\n",
    "        return\n",
    "    \n",
    "    print(f\"提取文字長度: {len(text)} 字符\")\n",
    "    \n",
    "    # 2. 生成摘要\n",
    "    print(\"2. 生成摘要...\")\n",
    "    summarizer = Summarizer()\n",
    "    summary_path, summary = summarizer.create_summary(text, filename)\n",
    "    \n",
    "    if summary:\n",
    "        print(f\"摘要長度: {len(summary)} 字符\")\n",
    "        print(f\"摘要內容: {summary[:200]}...\")\n",
    "    \n",
    "    # 3. 添加到向量數據庫\n",
    "    print(\"3. 添加到向量數據庫...\")\n",
    "    vector_store = VectorStore()\n",
    "    success = vector_store.add_document(text, filename)\n",
    "    \n",
    "    if success:\n",
    "        print(\"向量化處理成功\")\n",
    "    else:\n",
    "        print(\"向量化處理失敗\")\n",
    "        return\n",
    "    \n",
    "    # 4. 測試問答\n",
    "    print(\"4. 測試問答功能...\")\n",
    "    qa_service = QAService()\n",
    "    \n",
    "    test_question = \"這份文檔的主要內容是什麼？\"\n",
    "    result = qa_service.answer_question(test_question)\n",
    "    \n",
    "    print(f\"問題: {test_question}\")\n",
    "    print(f\"回答: {result['answer']}\")\n",
    "    print(f\"信心度: {result['confidence']}\")\n",
    "    \n",
    "    print(\"\\n完整流程測試完成！\")\n",
    "\n",
    "# 執行測試（請替換為實際的 PDF 檔案路徑）\n",
    "test_pdf_path = '../data/pdfs/test.pdf'\n",
    "test_full_pipeline(test_pdf_path)"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. 效能測試"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import time\n",
    "\n",
    "def benchmark_qa_service(num_queries=10):\n",
    "    \"\"\"\n",
    "    測試問答服務的效能\n",
    "    \"\"\"\n",
    "    qa_service = QAService()\n",
    "    \n",
    "    test_questions = [\n",
    "        \"什麼是機器學習？\",\n",
    "        \"深度學習的優勢是什麼？\",\n",
    "        \"如何改善模型效能？\",\n",
    "        \"什麼是過擬合？\",\n",
    "        \"如何選擇合適的演算法？\"\n",
    "    ]\n",
    "    \n",
    "    total_time = 0\n",
    "    successful_queries = 0\n",
    "    \n",
    "    for i in range(num_queries):\n",
    "        question = test_questions[i % len(test_questions)]\n",
    "        \n",
    "        start_time = time.time()\n",
    "        result = qa_service.answer_question(question)\n",
    "        end_time = time.time()\n",
    "        \n",
    "        query_time = end_time - start_time\n",
    "        total_time += query_time\n",
    "        \n",
    "        if result['confidence'] > 0:\n",
    "            successful_queries += 1\n",
    "        \n",
    "        print(f\"查詢 {i+1}: {query_time:.2f}s, 信心度: {result['confidence']}\")\n",
    "    \n",
    "    avg_time = total_time / num_queries\n",
    "    success_rate = successful_queries / num_queries * 100\n",
    "    \n",
    "    print(f\"\\n效能測試結果:\")\n",
    "    print(f\"平均查詢時間: {avg_time:.2f}s\")\n",
    "    print(f\"成功率: {success_rate:.1f}%\")\n",
    "    print(f\"總測試時間: {total_time:.2f}s\")\n",
    "\n",
    "# 執行效能測試\n",
    "benchmark_qa_service(5)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.8.5"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}

NameError: name 'null' is not defined